In [29]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

In [30]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer

from nltk import pos_tag
from collections import defaultdict

In [839]:
#somme global variables
prefixes = ['ante','anti','auto','co','com', 'con','contra', 'contro','de','dis','en','ex','extra','hetero','homo','hyper',
'il', 'im', 'in', 'ir','inter','intra','macro','non','omni','post','pre','pre','sub','sym', 'syn','tele','trans','tri','un','uni','up']

prefix_l2 = ['contra', 'contro','extra','hetero','inter','hyper','intra','macro','trans']

Noun_suffixes = ['acy','al','ance','ence','dom','er','ee','or','ism','ist','ity', 'ty','ment','ness','ship','sion','tion']
Verb_Suffixes  = ['ate','en','ify', 'fy','ize', 'ise']
Adjective_Suffixes = ['able', 'ible','al','ary','ed','ing', 'esque','ful','ic', 'ical','ish','ive','less','y']
Adverb_Suffixes = ['ly', 'ward', 'wise']
plural_suffixes = ['es', 's']
plural_sf = {'ies':'y' , 'ves':'f'}

suffixes = Noun_suffixes+Verb_Suffixes+Adjective_Suffixes+Adverb_Suffixes+plural_suffixes
Notusing_suffixes = ['ious', 'ous']

In [840]:
HANGMAN_URL = "https://www.trexsim.com/trexsim/hangman"

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.master_flag = 1
        self.current_dictionary = []
        self.wrong_guesses = []
        #self.strat3_flag = 1
        
    def guess(self, word): # word input example: "_ p p _ e "
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        clean_word = word[::2].replace("_",".") 
        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        nodup_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
                nodup_dictionary.append("".join(set(dict_word)))
                
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(nodup_dictionary)    
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()                   
        
        guess_letter = '!'
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break

        return guess_letter
    
    def guess_strat2(self, word, isstat2=True, m=0):
        clean_word = word[::2].replace("_",".")
        if isstat2:
            clean_word_temp = clean_word
            clean_word = clean_word_temp.strip('.')
            m = len(clean_word_temp) - len(clean_word)
            clean_word = '(.*)'+clean_word+'(.*)'
        # find length of passed word
        len_word = len(clean_word)
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        nodup_dictionary = []
        # iterate through all of the words in the old plausible dictionary
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) > m+1+ len_word or len(dict_word) < len_word:
                continue
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(clean_word,dict_word):
                new_dictionary.append(dict_word)
                nodup_dictionary.append("".join(set(dict_word)))   
                
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary        
        # count occurrence of all characters in possible word matches        
        full_dict_string = "".join(nodup_dictionary)
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common() 
        
        guess_letter = '!'
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                guess_letter = letter
                break  
        
        return guess_letter
        
    def word_clean_st3(self, word):
        clean_word = word[::2].replace("_",".")
        history_len = len(clean_word)
        # prefix cleaning code   
        pref = ""
        for pf in prefixes:
            if clean_word[:len(pf)] == pf and len(pref) < len(pf):  
                pref = pf
        '''for pf in prefix_l2:
            if re.match(clean_word[:len(pf)], pf) and len(clean_word[:len(pf)].replace(".", "")) => 3:
                pref = pf'''
        if pref != "":
            clean_word = clean_word[len(pref):]

        #final depluralization, as it can only occur in the last place
        suff = ""
        for sf in plural_sf:
            if clean_word[-len(sf):] == sf and len(suff) < len(sf):
                suff = sf
        if suff != "":
            clean_word = clean_word[:-len(suff)] + plural_sf[suff]
        #suffix cleaning code 
        suff = "./"
        while len(suff)>0:
            suff = ""
            for sf in suffixes:
                if clean_word[-len(sf):] == sf and len(suff) < len(sf):
                    suff = sf
            if suff != "":
                clean_word = clean_word[:-len(suff)]
        #final word
        word = " ".join(clean_word).replace(".","_") + ' '
        return word, history_len- len(clean_word)
    
    def guess_strat4(self, word):
        clean_word = word[::2].replace("_",".")
        guess_letter = '!'
        if clean_word.count('.') == 1 and clean_word[-1] == '.':
            if clean_word[-2] in ('l', 'c', 'h') and 'y' not in self.guessed_letters:
                guess_letter = 'y'
            elif clean_word[-2] == 'e' and 'd' not in self.guessed_letters:
                guess_letter = 'd'
            elif 's' not in self.guessed_letters:
                guess_letter = 's'
        if guess_letter =='!':
            sorted_letter_count = self.full_dictionary_common_letter_sorted
            for letter,instance_count in sorted_letter_count:
                if letter not in self.guessed_letters:
                    guess_letter = letter
                    break      
        else:
            print("Strat4 magic happened")
        return guess_letter
    
    def delete_irrelevent_words(self, letter):
        current_dictionary = self.current_dictionary
        #del_list = [word for word in current_dictionary if letter in word]
                
        final_dictionary = [wd for wd in current_dictionary if letter not in wd]
        return final_dictionary
    
    def build_dictionary(self,dictionary_file_location):
        print(" Building dictionary, will take 0.41 sec/1000 words")
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()

        lemmatizer = WordNetLemmatizer()
        snowball = SnowballStemmer(language='english')
        tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

        fuller_dictionary = []
        for word in full_dictionary:
            tk, tg = pos_tag([word])[0]
            lem =lemmatizer.lemmatize(tk, tag_dict.get(tg[0], wordnet.NOUN))
            stm = snowball.stem(word)

            if stm != word and word != lem:
                lem = stm
            sing = word
            for sf in plural_sf:
                if word[-len(sf):] == sf:
                    sing = word[:-len(sf)] + plural_sf[sf]
    
            fuller_dictionary += list(set([word,stm,lem, sing]))
        print(" Removing Duplicates")
        fuller_dictionary = list(dict.fromkeys(fuller_dictionary))
        return fuller_dictionary
    
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
                

In [841]:
api = HangmanAPI(access_token="16d04ed2733fd37b75934590e1186f", timeout=2000)

 Building dictionary, will take 0.41 sec/1000 words
 Removing Duplicates


In [842]:
#change the hidden letter here
def server_request(word, guess_letter, remainingtries):
    hidden = "unhomiletically"
    hidden = " ".join(hidden) +" "
    occurances = [i for i, letter in enumerate(hidden) if letter == guess_letter]
    word_lst = list(word)
    for pos in occurances:
        word_lst[pos] = guess_letter
    word = "".join(word_lst)
    res = {}
    if len(occurances) == 0:
        remainingtries -= 1
    status = "ongoing"
    if remainingtries == 0:
        dashes = [i for i, letter in enumerate(word) if letter == '_']
        if len(dashes) > 0:
            status = "failed"
        else:
            status = "success"
    res = {'status':status, 'tries_remains':remainingtries, 'word':word}
    return res


In [847]:
word = 'u n h o m i l e t i c a l l y'
word = '_ _ _ _ _ _ _ _ _ _ _ _ _ _ _'
remainingtries = 6
res = {'status':"started", 'tries_remains':remainingtries, 'word':word} 

In [851]:
print(res)

{'status': 'started', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ _ _ _ _ _'}


In [852]:
#def start_game(self, practice=True, verbose=True):
# reset guessed letters to empty set and current plausible dictionary to the full dictionary
api.guessed_letters = []
api.current_dictionary = api.full_dictionary
api.master_flag = 1
api.wrong_guesses = []   
#api.strat3_flag = 1

#response = api.request("/new_game", {"practice":practice})

word = res.get('word')
tries_remains = res.get('tries_remains')
print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(12324241, tries_remains, word))

Successfully start a new game! Game ID: 12324241. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _.


In [883]:
#while tries_remains>0:
# get guessed letter from user code
if api.master_flag ==1:
    guess_letter = api.guess(word)
    if guess_letter == '!':
        print("Strat 1 returned no solution, changing to strat 2.")
        api.master_flag = 2
        api.current_dictionary = api.full_dictionary

if api.master_flag ==2:
    guess_letter = api.guess_strat2(word,True)
    if guess_letter == '!':
        print("Strat 2 returned no solution, changing to strat 3.")
        api.master_flag = 3
        api.current_dictionary = api.full_dictionary

if api.master_flag ==3:
    new_word, m = api.word_clean_st3(word)
    print("new_word in strat3 reduction:", new_word)
    if new_word != word:
        guess_letter = api.guess_strat2(new_word, False,m)
        if  guess_letter == '!':
            print("Strat 3 returned no solution, changing to strat 4.")
            api.master_flag = 4
            api.current_dictionary = api.full_dictionary
    else:
        print("Strat 3 not useful now, changing to strat 4.")
        api.master_flag = 4
        api.current_dictionary = api.full_dictionary

if api.master_flag ==4:
    new_word, m = word, 0
    if word.count('_')/len(word) < 0.1:
        new_word, m = api.word_clean_st3(word)
    guess_letter = '!'
    if new_word != word:
        print("using new_word in clean3 reduction:", new_word)
        guess_letter = api.guess_strat2(new_word, False,m) #strat3
    if guess_letter == '!':
        print("using full_word in strat2:", new_word)
        api.current_dictionary = api.full_dictionary
        guess_letter = api.guess_strat2(word,True) #strat2
    if guess_letter == '!':
        print("Nothing worked!!")
        guess_letter = api.guess_strat4(word)

# append guessed letter to guessed letters field in hangman object
api.guessed_letters.append(guess_letter)

print("Guessing letter: {0}".format(guess_letter))
res = server_request(word, guess_letter, tries_remains)

print("Server response: ", res)

status = res.get('status')
tries_remains = res.get('tries_remains')
if status=="success":

    print("Successfully finished game: ")
    #return (True, api.master_flag)
elif status=="failed":
    
    print("Failed game: {0}. Because of: ")
    #return (False, api.master_flag)
elif status=="ongoing":
    if res.get('word') == word: # that is our guess is wrong
        api.wrong_guesses.append(guess_letter)
        if api.master_flag == 1:
            api.current_dictionary = api.delete_irrelevent_words(guess_letter)
        
    if len(api.current_dictionary) == 0 and api.master_flag < 4:
        api.master_flag += 1
        print("Current Scope is empty, changing to strat{0}".format(api.master_flag))
        api.current_dictionary = api.full_dictionary
    # uppdating the word
    word = res.get('word')

using new_word in clean3 reduction: h o m i l e t i c a l l _ 
using full_word in strat2: h o m i l e t i c a l l _ 
Nothing worked!!
Strat4 magic happened
Guessing letter: y
Server response:  {'status': 'ongoing', 'tries_remains': 2, 'word': 'u n h o m i l e t i c a l l y'}


In [884]:
print(api.guessed_letters)

print(api.wrong_guesses)

['i', 't', 'n', 'l', 'e', 'a', 'o', 'r', 's', 'c', 'u', 'd', 'p', 'm', 'h', 'y']
['r', 's', 'd', 'p']


In [638]:
api.full_dictionary

328457

In [ ]:
lemmatizer = WordNetLemmatizer()
snowball = SnowballStemmer(language='english')
tag_dict = {"J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV}

tk, tg = pos_tag([word])[0]
lem =lemmatizer.lemmatize(tk, tag_dict.get(tg[0], wordnet.NOUN))
stm = snowball.stem(word)

In [349]:
word = 'n i _ e r n a i s e '

In [357]:
new_word, m = api.word_clean_st3(word)
new_word, m 

('n i _ e r n a ', 3)

In [351]:
if False:
    clean_word_temp = word[::2].replace("_",".")
    clean_word = clean_word_temp.strip('.')
    m = len(clean_word_temp) - len(clean_word)
    # find length of passed word
    len_word = len(clean_word)
    clean_word = '(.*)'+clean_word+'(.*)'
else:
    #word ,m = self.word_clean_st3(word)
    clean_word = new_word[::2].replace("_",".")
    len_word = len(clean_word)
    
print(clean_word, m)


ni.erna 3


In [893]:
clean_word = '(.*)pluc.er(.*)'
len_word = 14

In [894]:
# grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
current_dictionary = api.full_dictionary
new_dictionary = []
nodup_dictionary = []
# iterate through all of the words in the old plausible dictionary
for dict_word in current_dictionary:
    # continue if the word is not of the appropriate length
    if len(dict_word) > m+1+ len_word or len(dict_word) < len_word:
        continue
    # if dictionary word is a possible match then add it to the current dictionary
    if re.match(clean_word,dict_word):
        new_dictionary.append(dict_word)
        nodup_dictionary.append("".join(set(dict_word)))   



In [895]:
# overwrite old possible words dictionary with updated version
#self.current_dictionary = new_dictionary        
# count occurrence of all characters in possible word matches        
full_dict_string = "".join(nodup_dictionary)
c = collections.Counter(full_dict_string)
sorted_letter_count = c.most_common() 

In [896]:
new_dictionary

[]

In [814]:
sorted_letter_count

[('m', 25),
 ('u', 25),
 ('p', 25),
 ('t', 25),
 ('i', 25),
 ('l', 25),
 ('e', 20),
 ('n', 16),
 ('a', 14),
 ('o', 13),
 ('c', 12),
 ('r', 8),
 ('s', 8),
 ('d', 6),
 ('v', 5),
 ('g', 4),
 ('h', 4),
 ('y', 4),
 ('x', 2),
 ('b', 2),
 ('f', 1)]

In [445]:
clean_word = word[::2].replace("_",".")
if True:
    clean_word_temp = clean_word
    clean_word = clean_word_temp.strip('.')
    m = len(clean_word_temp) - len(clean_word)

# find length of passed word
len_word = len(clean_word)

In [446]:
plural_suffixes

('a..i.io..n', 6)

In [584]:
dictionary_file_location = "words_250000_train.txt"
text_file = open(dictionary_file_location,"r")
full_dictionary = text_file.read().splitlines()
text_file.close()

In [604]:
plurals = []
plural_sf = {'ies':'y' , 'ves':'f'}
for word in full_dictionary:
    sing = word
    for sf in plural_sf:
        if word[-len(sf):] == sf:
            sing = plurals.append(word[:-len(sf)] + plural_suffixes[sf])
        
    
        

ies
ies
ies
ies
ves
ies
ies
ies
ves
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ves
ies
ies
ies
ves
ies
ves
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ves
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ves
ves
ves
ves
ves
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ies
ves
ies
ies
ies


In [593]:
sf = "ies"
clean_word = "dqdaasfsies"
clean_word[-len(sf):]

'ies'